In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# Load Churn OD data
churn_od = pd.read_csv('Churn_OD.txt', delimiter='|', encoding= 'unicode_escape')
churn_user_profile = pd.read_csv('Churn_UsersProfile.txt', delimiter = "|", encoding= 'unicode_escape')

In [ ]:
import numpy as np

churn_user_profile_copy = churn_user_profile.copy()

tmp = churn_user_profile_copy.merge(
    churn_user_profile_copy, on=[
        'Region_of_Origin', 
        'District_of_Origin', 
        'County_of_Origin',
        'GenderDescription',  # Replace by aggregate_variables 
        'AgeClassDescription'
    ], how='left'
)

# Keep rows where period_x is different from period_y, and period_x is in 2019
tmp = tmp[tmp.Period_x != tmp.Period_y]
tmp = tmp[tmp.Period_x == 'Sep-19 to Feb-20']

tmp['Relative_Change_AverageBusUsers'] = (tmp['Average_BusUsers_per_Day_y'] - tmp['Average_BusUsers_per_Day_x'])/(tmp['Average_BusUsers_per_Day_x'])
tmp['Relative_Change_AverageBusUsers_Abs'] = np.abs((tmp['Average_BusUsers_per_Day_y'] - tmp['Average_BusUsers_per_Day_x']))/(tmp['Average_BusUsers_per_Day_x'])

In [ ]:
churn_od_std_demand = churn_od.groupby(['Region_of_Origin', 'District_of_Origin', 'County_of_Origin'])['Demand_weight'].std().reset_index()

daily_users_changes = tmp
churn_od_std_demand.rename(columns = {'Demand_weight': 'std_demand_weight'}, inplace=True)
tmp = daily_users_changes.groupby('County_of_Origin')['Relative_Change_AverageBusUsers'].mean().reset_index()
tmp.merge(churn_od_std_demand)


In [ ]:
#ADICIONAR 2 COLUNAS COM LATITUDE E LONGITUDE

l=pd.read_csv('pt.csv', delimiter=',')
tmp["Lat"]=l["lat"]
tmp["Long"]=l["lng"]

In [ ]:
tmp

In [ ]:
!pip install plotly
#!pip install chart_studio

#import chart_studio
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pandas as pd

# setting user, api key and access token
plotly.tools.set_credentials_file(username='JOANAMORGADO', api_key='xJI1ZWUk2F195acmdtOF')
mapbox_access_token = 'pk.eyJ1Ijoiam9hbmFtb3JnYWRvIiwiYSI6ImNrbjh2emt3eDBlNmYydnBkOWFrODR4b3oifQ.KnJ1gWnG0dbmsjrDol609A'

In [ ]:
data = []
for relative in tmp['Relative_Change_AverageBusUsers']:
    county_data = dict(
            lat = tmp.loc[tmp['Relative_Change_AverageBusUsers'] == relative, 'Lat'],
            lon = tmp.loc[tmp['Relative_Change_AverageBusUsers'] == relative, 'Long'],
            name = relative,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(county_data)


In [ ]:
data

In [ ]:
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered,
        #VER - QUAL É O CONCELHO QUE QUEREMOS NO CENTRO (LONG E LATITUDE) Lisboa por agora
        center = dict(
            lat = 38.7452,
            lon = -9.1604
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 3,
        # default map style
        style = 'dark'
    )
)

In [ ]:
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.75,
        xanchor = 'left',
        y = 0.05,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # 
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.01,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# assign the list of dictionaries to the layout dictionary
layout['updatemenus'] = updatemenus

In [ ]:
figure = dict(data = data, layout = layout)
py.iplot(figure)
#(figure, filename = 'put_your_name_here')